# Bollinger Bands indicator

In [1]:
import pandas as pd
import numpy as np

# Load the 1-minute OHLCV data
df = pd.read_csv('ohlc_1m_data_last_day.csv', parse_dates=['timestamp'])

# Calculate Bollinger Bands
def calculate_bollinger_bands(df, window=20, no_of_std=2):
    df['MA'] = df['close'].rolling(window=window).mean()
    df['BB_Upper'] = df['MA'] + (df['close'].rolling(window=window).std() * no_of_std)
    df['BB_Lower'] = df['MA'] - (df['close'].rolling(window=window).std() * no_of_std)
    return df

# Generate BB signals
def generate_bb_signals(df):
    df['BB_Signals'] = np.nan  # Default to NaN for no signal
    df.loc[df['close'] < df['BB_Lower'], 'BB_Signals'] = 'Buy'  
    df.loc[df['close'] > df['BB_Upper'], 'BB_Signals'] = 'Sell'  
    return df

# Apply Bollinger Bands calculation and generate signals on 1-minute data
df = calculate_bollinger_bands(df)
df = generate_bb_signals(df)

# Resample to 4-hour intervals
df_resampled = df.resample('4H', on='timestamp').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'BB_Upper': 'last',
    'BB_Lower': 'last',
    'BB_Signals': 'last'  
})

# Generate buy/sell signals for the 4-hour data
df_resampled['Signal'] = np.where(df_resampled['close'] < df_resampled['BB_Lower'], 'Buy', 
                                  np.where(df_resampled['close'] > df_resampled['BB_Upper'], 'Sell', np.nan))

# Convert NaN values in Signal column based on BB_Signals and ensure integer values
df_resampled['Signal'] = df_resampled.apply(lambda row: 1 if row['BB_Signals'] == 'Buy' else 
                                            (-1 if row['BB_Signals'] == 'Sell' else np.nan), axis=1)

# Convert the Signal column to integers
df_resampled['Signal'] = df_resampled['Signal'].astype('Int64')  # Use 'Int64' to handle NaN values

# Save the resampled data with signals
df_resampled.to_csv('ohlc_4h_bb_signals.csv', index=False)

print(df_resampled.head())


C:\Users\MTariq\AppData\Local\Temp\ipykernel_1152\3019690448.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['close'] < df['BB_Lower'], 'BB_Signals'] = 'Buy'
C:\Users\MTariq\AppData\Local\Temp\ipykernel_1152\3019690448.py:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('4H', on='timestamp').agg({


                        open     high      low    close     BB_Upper  \
timestamp                                                              
2020-01-01 00:00:00  7195.24  7245.00  7175.46  7225.01  7231.275725   
2020-01-01 04:00:00  7225.00  7236.27  7199.11  7209.83  7214.145374   
2020-01-01 08:00:00  7209.83  7237.73  7180.00  7197.20  7237.216457   
2020-01-01 12:00:00  7197.20  7255.00  7196.15  7234.19  7244.079857   
2020-01-01 16:00:00  7234.20  7249.99  7214.00  7229.48  7235.314704   

                        BB_Lower BB_Signals  Signal  
timestamp                                            
2020-01-01 00:00:00  7220.990275        Buy       1  
2020-01-01 04:00:00  7200.996626        Buy       1  
2020-01-01 08:00:00  7179.881543        Buy       1  
2020-01-01 12:00:00  7219.126143       Sell      -1  
2020-01-01 16:00:00  7226.242296       Sell      -1  
